# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

In [131]:
import pandas as pd
import xlwings as xw
import xlsxwriter

## Лабораторная работа 7

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [139]:
reviews_df = pd.read_csv(
    'data/reviews_sample.csv',
    sep=',',
    index_col=0,
    parse_dates=['date']
)
reviews_df.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [140]:
recipes_df = pd.read_csv(
    'data/recipes_sample_with_filled_nsteps.csv',
    sep=',',
    usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'],
    parse_dates=['submitted']
)
recipes_df.head()

,name,id,minutes,submitted,description,n_ingredients
0,george s at the cove black bean soup,44123,90,2002-10-25,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,2003-07-26,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,2002-08-29,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [213]:
reviews_sample = reviews_df.sample(frac=0.05)
recipes_sample = recipes_df.sample(frac=0.05)

recipes_len = recipes_sample.shape[0]
reviews_len = reviews_sample.shape[0]

writer = pd.ExcelWriter('data/recipes.xlsx', engine='xlsxwriter')
reviews_sample.to_excel(writer, sheet_name='Отзывы', index=False)
recipes_sample.to_excel(writer, sheet_name='Рецепты', index=False)

In [214]:
workbook = writer.book
recipes_sheet_w = writer.sheets['Рецепты']
reviews_sheet_w = writer.sheets['Отзывы']

In [215]:
bg_green = workbook.add_format({'bg_color': '#228b22'})

bg_yellow = workbook.add_format({'bg_color': '#f7f21a'})

bg_red = workbook.add_format({'bg_color': '#FFC7CE'})

recipes_sheet_w.conditional_format(f'C2:C{recipes_len}',
                                        {'type': 'cell',
                                         'criteria': '<',
                                         'value': 5,
                                         'format': bg_green})

recipes_sheet_w.conditional_format(f'C2:C{recipes_len}',
                                        {'type': 'cell',
                                         'criteria': 'between',
                                         'minimum': 5,
                                         'maximum': 10,
                                         'format': bg_yellow})


recipes_sheet_w.conditional_format(f'C2:C{recipes_len}',
                                        {'type': 'cell',
                                         'criteria': '>',
                                         'value': 10,
                                         'format': bg_red})

reviews_sheet_w.conditional_format(f'$A$2:$E${reviews_len}',
                                        {'type': 'formula',
                                         'criteria': '=OR($D2<0,$D2>5,COUNTIF(Рецепты!$B2:$B1500,"="&$B2)<1)',
                                         'format': bg_red})

In [216]:
workbook.close()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [217]:
wb = xw.Book('data/recipes.xlsx')
recipes_sample['second_assign'] = recipes_sample['minutes'] * 60
recipes_sheet = wb.sheets['Рецепты']
recipes_sheet.range("g1").options(index=False).value = recipes_sample['second_assign']

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [218]:
recipes_sheet['H1'].value = 'seconds_formula'
recipes_sheet.range(f'H2:H{recipes_sample.shape[0]}').formula = '=C2 * 60'

5. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [219]:
recipes_sheet['I1'].value = 'n_reviews'
recipes_sheet.range(f'I2:I{recipes_sample.shape[0]}').formula = '=COUNTIF(Отзывы!B:B, "="&B2)'

6. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [220]:
recipes_sheet.range('G:G').api.Font.Bold = True
recipes_sheet.range('H:H').api.Font.Bold = True
recipes_sheet.range('I:I').api.Font.Bold = True

recipes_sheet.range('G:G').autofit()
recipes_sheet.range('H:H').autofit()
recipes_sheet.range('I:I').autofit()

recipes_sheet.range('G:G').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
recipes_sheet.range('H:H').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
recipes_sheet.range('I:I').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

7. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [94]:
# Добавлено условное форматирование в пункте 2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
# Добавлено условное форматирование в пункте 2